In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.regression_models as RegressionModels
import core.assignment_models as AssignmentModels
import core
import keras
import core.regression_models.RegressionTransformer as RegressionTransformer

plt.rcParams["font.size"] = 14

PLOTS_DIR = f"plots/regresion_transformer/"
MODEL_DIR = f"models/regresion_transformer/"
CONFIG_PATH = "../config/workspace_config_high_level_features.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)

with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

data_config = DataProcessor.load_from_npz(
    data_configs["npz_datapath"]["nominal"], max_events=4000000
)

X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)

2025-11-18 16:01:17.491215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763478077.513169  109450 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763478077.520350  109450 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763478077.537601  109450 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763478077.537621  109450 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763478077.537624  109450 computation_placer.cc:177] computation placer alr

In [2]:
reload(RegressionTransformer)
RegressionTransformer = RegressionTransformer.FeatureConcatTransformer(data_config)
# RegressionTransformer.load_model(MODEL_DIR + "model.keras")

In [ ]:
RegressionTransformer.build_model(
    hidden_dim=256,
    num_heads=8,
    central_transformer_stack_size=4,
    regression_transformer_stack_size=4,
    dropout_rate=0.1,
)
RegressionTransformer.compile_model(
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4),
    loss={
        "regression": core.utils.MagnitudeDirectionLoss(),
        "assignment": core.utils.AssignmentLoss(),
    },
    metrics={
        "regression": core.utils.RegressionDeviation(),
        "assignment": core.utils.AssignmentAccuracy(),
    },
    loss_weights={
        "regression": 1.0,
        "assignment": 1.0,
    },
)


I0000 00:00:1763478246.757523  109450 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


In [ ]:
RegressionTransformer.train_model(
    X_train,
    y_train,
    batch_size=1024,
    copy_data=True,
    epochs=50,
)

Epoch 1/50


I0000 00:00:1763478278.584461  113529 service.cc:152] XLA service 0x7fd11c01a490 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763478278.584490  113529 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-11-18 16:04:39.753478: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-18 16:04:41.002186: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1763478284.334394  113529 cuda_dnn.cc:529] Loaded cuDNN version 91500
2025-11-18 16:05:13.938182: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_5', 44 bytes spill stores, 44 bytes spill

   1/2813 ━━━━━━━━━━━━━━━━━━━━ 50:11:27 64s/step - assignment_accuracy: 0.2178 - assignment_loss: 0.0076 - loss: 2.0008 - regression_deviation: 1.0000 - regression_loss: 1.9932

I0000 00:00:1763478314.629464  113529 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2812/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - assignment_accuracy: 0.3438 - assignment_loss: 0.0047 - loss: 1.5615 - regression_deviation: 1.0003 - regression_loss: 1.5569

2025-11-18 16:12:00.818790: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-18 16:12:33.874402: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 56 bytes spill stores, 56 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_21', 48 bytes spill stores, 48 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_16', 124 bytes spill stores, 124 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_11', 44 bytes spill stores, 44 bytes spill loads



2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - assignment_accuracy: 0.3439 - assignment_loss: 0.0047 - loss: 1.5615 - regression_deviation: 1.0003 - regression_loss: 1.5569

2025-11-18 16:12:36.815193: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-18 16:12:53.087541: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


2813/2813 ━━━━━━━━━━━━━━━━━━━━ 525s 164ms/step - assignment_accuracy: 0.3439 - assignment_loss: 0.0047 - loss: 1.5615 - regression_deviation: 1.0003 - regression_loss: 1.5569 - val_assignment_accuracy: 0.4779 - val_assignment_loss: 0.0033 - val_loss: 1.4481 - val_regression_deviation: 1.0013 - val_regression_loss: 1.4449
Epoch 2/50
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 417s 148ms/step - assignment_accuracy: 0.4515 - assignment_loss: 0.0036 - loss: 1.4609 - regression_deviation: 1.0527 - regression_loss: 1.4572 - val_assignment_accuracy: 0.4810 - val_assignment_loss: 0.0032 - val_loss: 0.8085 - val_regression_deviation: 3.7098 - val_regression_loss: 0.8054
Epoch 3/50
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 416s 148ms/step - assignment_accuracy: 0.4586 - assignment_loss: 0.0035 - loss: 0.8577 - regression_deviation: 3.7876 - regression_loss: 0.8543 - val_assignment_accuracy: 0.4890 - val_assignment_loss: 0.0031 - val_loss: 0.7616 - val_regression_deviation: 3.9595 - val_regression_loss: 0.7586
Epoch 4/5

In [ ]:
RegressionTransformer.save_model(MODEL_DIR + "model.keras")

In [ ]:
AssignmentTransformer = AssignmentModels.FeatureConcatTransformer(
    data_config, name=r"Assignment Transformer + $\nu$-Flows"
)
AssignmentTransformer.load_model(MODEL_DIR + "model.keras")

In [ ]:
import core.assignment_models.BaselineAssignmentMethods as BaselineMethods
import core.reconstruction as Evaluation

reload(Evaluation)
reload(BaselineMethods)
delta_r_assigner = BaselineMethods.DeltaRAssigner(data_config)
chi_square_true_nu = BaselineMethods.ChiSquareAssigner(
    data_config,
    top_mass=173.5e3,
)
ground_truth_assigner = Evaluation.GroundTruthReconstructor(
    data_config, name="Perfect Reconstructor"
)
evaluator = Evaluation.ReconstructionEvaluator(
    [
        chi_square_true_nu,
        RegressionTransformer,
        AssignmentTransformer,
        ground_truth_assigner,
    ],
    X_val,
    y_val,
)

In [ ]:
fig, ax = evaluator.plot_all_accuracies()
fig.savefig(PLOTS_DIR + "all_accuracies.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_top_mass_resolution(
    feature_data_type="non_training",
    feature_name="truth_ttbar_mass",
    fancy_feature_label=r"$m(t\overline{t})$ [GeV]",
    xlims=(340e3, 800e3),
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)
fig.savefig(PLOTS_DIR + "top_mass_resolution_comparison.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_ttbar_mass_resolution(
    feature_data_type="non_training",
    feature_name="truth_ttbar_mass",
    fancy_feature_label=r"$m(t\overline{t})$ [GeV]",
    xlims=(340e3, 800e3),
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)

fig.savefig(PLOTS_DIR + "ttbar_mass_resolution_comparison.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_complementarity(
    feature_data_type="non_training",
    feature_name="truth_ttbar_mass",
    fancy_feature_label=r"$m(t\overline{t})$ [GeV]",
    xlims=(340e3, 800e3),
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)

fig.savefig(PLOTS_DIR + "binned_complementarity_ttbar_mass.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_accuracy(
    feature_data_type="non_training",
    feature_name="truth_ttbar_mass",
    fancy_feature_label=r"$m(t\overline{t})$ [GeV]",
    xlims=(340e3, 800e3),
    bins=10,
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)
fig.savefig(PLOTS_DIR + "binned_accuracy_ttbar_mass.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_accuracy(
    feature_data_type="non_training",
    feature_name="truth_ttbar_pt",
    fancy_feature_label=r"$p_T(t\overline{t})$ [GeV]",
    xlims=(0, 400e3),
    bins=10,
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(0, 400e3)

fig.savefig(PLOTS_DIR + "binned_accuracy_ttbar_pT.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_accuracy(
    feature_data_type="non_training",
    feature_name="truth_initial_parton_num_gluons",
    fancy_feature_label=r"Initial State",
    xlims=(0, 3),
    bins=3,
)
ax.set_xticks([0.5, 1.5, 2.5])
ax.set_xticklabels([r"$qq\to tt$", r"$qg\to tt$", r"$gg\to tt$"])
fig.savefig(PLOTS_DIR + "binned_accuracy_initial_state.pdf")

In [ ]:
fig, ax = evaluator.plot_binned_accuracy(
    feature_data_type="non_training",
    feature_name="N_jets",
    fancy_feature_label=r"$\# \text{jets}$",
    xlims=(2, data_configs["LoadConfig"]["max_jets"] + 1),
    bins=data_configs["LoadConfig"]["max_jets"] - 1,
)
ax.set_xticks([i + 0.5 for i in range(2, data_configs["LoadConfig"]["max_jets"] + 1)])
ax.set_xticklabels([i for i in range(2, data_configs["LoadConfig"]["max_jets"] + 1)])
fig.savefig(PLOTS_DIR + "binned_accuracy_N_jets.pdf")

In [ ]:
fig, ax = evaluator.plot_confusion_matrices()
fig.savefig(PLOTS_DIR + "confusion_matrices.pdf")

In [ ]:
# Plot neutrino component deviation histograms
fig, axes = evaluator.plot_neutrino_component_deviations(
    bins=50,
    xlims=(0, 5),  # Optional: limit x-axis range
    figsize=(20, 15),
    component_labels=["$p_x$", "$p_y$", "$p_z$"],
)

In [ ]:
# Plot overall relative deviation distribution
fig, ax = evaluator.plot_all_neutrino_deviations()